In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 
import requests
from json import load
import json


## Understanding the JSON schema from DataBase

In [2]:
# Loading Schema
with open('Agri_schema.json', "r") as sc:
    schema = load(sc)
    
    pretty_json = json.dumps(schema, indent=3)
    print(pretty_json)

{
   "title": "agri-environmental-indicators-emissions-by-sector",
   "type": "object",
   "oneOf": [
      {
         "$ref": "#/definitions/agri-environmental-indicators-emissions-by-sector"
      }
   ],
   "definitions": {
      "agri-environmental-indicators-emissions-by-sector": {
         "properties": {
            "records": {
               "type": "array",
               "items": {
                  "$ref": "#/definitions/agri-environmental-indicators-emissions-by-sector_records"
               }
            }
         }
      },
      "agri-environmental-indicators-emissions-by-sector_records": {
         "properties": {
            "fields": {
               "type": "object",
               "properties": {
                  "year": {
                     "type": "string",
                     "format": "date",
                     "title": "Year",
                     "description": ""
                  },
                  "area": {
                     "type": "string",


# Open JSON file - Download file from link in Readme or Resources

In [3]:
json_file ="agri-environmental-indicators-emissions-by-sector.json"

In [4]:
entire_file_df = pd.read_json(json_file)
entire_file_df.head()

,datasetid,recordid,fields,record_timestamp
0,agri-environmental-indicators-emissions-by-sector,5500592b928d56fc0da60501fa364fa5c132b79a,"{'year': '2014', 'element': 'Emissions (F-gase...",2021-11-07T05:29:57.595-06:00
1,agri-environmental-indicators-emissions-by-sector,6b85b6ddf5ef6956602f5794450b16671561f5ff,"{'year': '2018', 'element': 'Emissions (F-gase...",2021-11-07T05:29:57.595-06:00
2,agri-environmental-indicators-emissions-by-sector,7315471f71541c41f2a5349f7bb4b522a051931f,"{'year': '1991', 'element': 'Emissions Share (...",2021-11-07T05:29:57.595-06:00
3,agri-environmental-indicators-emissions-by-sector,825a93245b1d222469a12604a1651cc34a63bd52,"{'year': '1999', 'element': 'Emissions Share (...",2021-11-07T05:29:57.595-06:00
4,agri-environmental-indicators-emissions-by-sector,3f4119c9755d2fddac4126a472f149131a6bace1,"{'year': '2012', 'element': 'Emissions Share (...",2021-11-07T05:29:57.595-06:00


In [5]:
fields_df = entire_file_df.loc[:, "fields"]
fields_df.head()

0    {'year': '2014', 'element': 'Emissions (F-gase...
1    {'year': '2018', 'element': 'Emissions (F-gase...
2    {'year': '1991', 'element': 'Emissions Share (...
3    {'year': '1999', 'element': 'Emissions Share (...
4    {'year': '2012', 'element': 'Emissions Share (...
Name: fields, dtype: object

### Make a Usable DataFrame

In [6]:
ent_json = pd.json_normalize(fields_df)
ent_json

,year,element,unit,area,item,value
0,2014,Emissions (F-gases),kilotonnes,Mongolia,Waste - food systems,0.0000
1,2018,Emissions (F-gases),kilotonnes,Mongolia,Waste - food systems,0.0000
2,1991,Emissions Share (CO2eq) (AR5),%,Mongolia,Food Household Consumption,0.9332
3,1999,Emissions Share (CO2eq) (AR5),%,Mongolia,Food Household Consumption,0.3126
4,2012,Emissions Share (CO2eq) (AR5),%,Mongolia,Food Household Consumption,0.6152
...,...,...,...,...,...,...
1683943,2000,Emissions (N2O),kilotonnes,South America,Waste - food systems,9.9872
1683944,2011,Emissions (N2O),kilotonnes,South America,Waste - food systems,13.0647
1683945,2016,Emissions (N2O),kilotonnes,South America,Waste - food systems,12.9916
1683946,1992,Emissions (F-gases),kilotonnes,South America,Waste - food systems,0.0000


### Selecting Major 50 countries Representating Emissions DataBase 

In [7]:

counrty_list = ['United States', 'Mexico', 'Brazil', 'Columbia', 'Venezuela','Chile','Peru','India','Bangladesh','Sri Lanka',
                'Myanmar', 'Pakistan', 'Russia', 'Ukraine', 'Kazakhstan', 'Azerbaijan', 'Armenia', 'Afghanistan', 'Uzbekistan',
                'South Africa', 'Nigeria', 'Ghana', 'Morocco', 'Egypt', 'Kenya', 'China', 'Japan', 'Hong Kong', 'Singapore', 
                'Thailand', 'Malaysia', 'Korea', 'Viet Nam', 'United Kingdom', 'France', 'Germany', 'Norway', 'Italy', 
                'Spain', 'Portugal', 'Sweden', 'Turkey', 'Greece', 'Saudi Arabia', 'Bahrain', 'Syrian Arab Republic', 
                'Iraq', 'Qatar', 'United Arab Emirates', 'Kuwait']

In [8]:
Agri_emis_cntry = ent_json.loc[ent_json["area"].isin(counrty_list)]
Agri_emis_cntry

,year,element,unit,area,item,value
697,2000,Emissions Share (CO2),%,Mexico,Energy - food systems,12.4978
698,2009,Emissions Share (CO2),%,Mexico,Energy - food systems,12.8555
699,2013,Emissions Share (CO2),%,Mexico,Energy - food systems,11.8992
700,2016,Emissions Share (CO2),%,Mexico,Energy - food systems,12.5585
701,2018,Emissions Share (CO2),%,Mexico,Energy - food systems,13.1745
...,...,...,...,...,...,...
1680933,1991,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,4648.9348
1680934,1996,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,4678.1061
1680935,1997,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,4646.0924
1680936,2016,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,1923.3412


### Cleaning Data 

In [9]:
Agri_emis_cntry = Agri_emis_cntry.dropna(how='any')
Agri_emis_cntry.reset_index(drop = True, inplace = True)
Agri_emis_cntry

,year,element,unit,area,item,value
0,2000,Emissions Share (CO2),%,Mexico,Energy - food systems,12.4978
1,2009,Emissions Share (CO2),%,Mexico,Energy - food systems,12.8555
2,2013,Emissions Share (CO2),%,Mexico,Energy - food systems,11.8992
3,2016,Emissions Share (CO2),%,Mexico,Energy - food systems,12.5585
4,2018,Emissions Share (CO2),%,Mexico,Energy - food systems,13.1745
...,...,...,...,...,...,...
302723,1991,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,4648.9348
302724,1996,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,4678.1061
302725,1997,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,4646.0924
302726,2016,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,1923.3412


### Understanding the various industry Sectors used in Emissions data 

In [10]:
Agri_emis_sector = Agri_emis_cntry['item'].value_counts()
Agri_emis_sector_df = pd.DataFrame(Agri_emis_sector)
Agri_emis_sector_df

,item
Food Transport,12593
Farm-gate emissions,12593
On-farm electricity use,12593
All sectors without LULUCF,12593
Food Household Consumption,12593
Waste - food systems,12593
Food Waste Disposal,12593
IPCC Agriculture,12593
Energy,12593
Emissions on agricultural land,12593


### Pushing Industry / Sectors out to CSV 

In [11]:
Agri_emis_sector_df.to_csv("Agri_emis_by_Sector.csv", index= True, header = True)

### Understanding the various compounds used in emssions database

In [12]:
Agri_emis_gases = Agri_emis_cntry['element'].value_counts()
Agri_emis_gases_df = pd.DataFrame(Agri_emis_gases)
Agri_emis_gases_df

,element
Emissions Share (CO2eq) (AR5),32239
Emissions (CO2eq) (AR5),32239
Emissions (F-gases),31650
Emissions Share (N2O),31139
Emissions (N2O),31139
Emissions Share (CO2),29586
Emissions (CO2),29586
Emissions (CH4),29098
Emissions Share (CH4),29098
Emissions Share (F-gases),26954


# Since "Emission Share" has a unit of % whereas normal emissions are measured in kilotonnes it is thought that 2 separate dataframes would be beneficial for loading into Database for Query purposes - one with % emissions and other with actual value

### Making the 1st Dataframe that represents % emissions of the 50 selected countries

In [13]:
Agri_em_percent = Agri_emis_cntry.loc[(Agri_emis_cntry["element"] == "Emissions Share (CO2eq) (AR5)") | 
                                      (Agri_emis_cntry["element"] == "Emissions Share (N2O)") |
                                        (Agri_emis_cntry["element"] == "Emissions Share (CO2)") |
                                      (Agri_emis_cntry["element"] == "Emissions Share (CH4)") |
                                       (Agri_emis_cntry["element"] == "Emissions Share (F-gases)"), :]
Agri_em_percent.reset_index(drop = True, inplace = True)
Agri_em_percent
                                   
                                      

,year,element,unit,area,item,value
0,2000,Emissions Share (CO2),%,Mexico,Energy - food systems,12.4978
1,2009,Emissions Share (CO2),%,Mexico,Energy - food systems,12.8555
2,2013,Emissions Share (CO2),%,Mexico,Energy - food systems,11.8992
3,2016,Emissions Share (CO2),%,Mexico,Energy - food systems,12.5585
4,2018,Emissions Share (CO2),%,Mexico,Energy - food systems,13.1745
...,...,...,...,...,...,...
149011,2001,Emissions Share (N2O),%,Sri Lanka,LULUCF,0.0000
149012,2004,Emissions Share (N2O),%,Sri Lanka,LULUCF,0.0259
149013,2012,Emissions Share (N2O),%,Sri Lanka,LULUCF,0.0614
149014,2016,Emissions Share (N2O),%,Sri Lanka,LULUCF,0.0475


### Renaming & Organizing columns

In [14]:
Rename_Agri_em_percent = Agri_em_percent.rename(columns = {"area": "Country", "value":"Percent_Glb_Agri_Emission", "element":"GH-gases", "item": "Agri_Industry_Sector" })
Org_agri_emi_per = Rename_Agri_em_percent[["year", "Country", "GH-gases", "Agri_Industry_Sector", "Percent_Glb_Agri_Emission", "unit"]]
Org_agri_emi_per

,year,Country,GH-gases,Agri_Industry_Sector,Percent_Glb_Agri_Emission,unit
0,2000,Mexico,Emissions Share (CO2),Energy - food systems,12.4978,%
1,2009,Mexico,Emissions Share (CO2),Energy - food systems,12.8555,%
2,2013,Mexico,Emissions Share (CO2),Energy - food systems,11.8992,%
3,2016,Mexico,Emissions Share (CO2),Energy - food systems,12.5585,%
4,2018,Mexico,Emissions Share (CO2),Energy - food systems,13.1745,%
...,...,...,...,...,...,...
149011,2001,Sri Lanka,Emissions Share (N2O),LULUCF,0.0000,%
149012,2004,Sri Lanka,Emissions Share (N2O),LULUCF,0.0259,%
149013,2012,Sri Lanka,Emissions Share (N2O),LULUCF,0.0614,%
149014,2016,Sri Lanka,Emissions Share (N2O),LULUCF,0.0475,%


### Deleting rows with 0 emmisions and Finalizing DataFrame for loading into SQL

In [15]:
del Org_agri_emi_per['unit']
final_agri_em_per_df = Org_agri_emi_per.loc[(Org_agri_emi_per["Percent_Glb_Agri_Emission"]>0), :]
final_agri_em_per_df = final_agri_em_per_df.sort_values("year")
final_agri_em_per_df.reset_index (drop =True, inplace = True)
final_agri_em_per_df


,year,Country,GH-gases,Agri_Industry_Sector,Percent_Glb_Agri_Emission
0,1990,Egypt,Emissions Share (CO2eq) (AR5),Energy,66.7577
1,1990,Japan,Emissions Share (CO2eq) (AR5),All sectors without LULUCF,106.3107
2,1990,Japan,Emissions Share (CH4),All sectors with LULUCF,100.0000
3,1990,Japan,Emissions Share (N2O),Food Packaging,0.2160
4,1990,Peru,Emissions Share (N2O),Food Waste Disposal,1.9951
...,...,...,...,...,...
115243,2019,Azerbaijan,Emissions Share (CO2),On-farm electricity use,1.6114
115244,2019,Portugal,Emissions Share (F-gases),Pre- and post- production,3.0105
115245,2019,Azerbaijan,Emissions Share (CO2eq) (AR5),On-farm electricity use,1.0351
115246,2019,Bangladesh,Emissions Share (CH4),IPPU,0.0001


In [16]:
final_agri_em_per_df.count()

year                         115248
Country                      115248
GH-gases                     115248
Agri_Industry_Sector         115248
Percent_Glb_Agri_Emission    115248
dtype: int64

### Making 2nd Dataframe which represents actual values of emissions in Kilotonnes

In [17]:
Agri_em_actual = Agri_emis_cntry.loc[(Agri_emis_cntry["element"] == "Emissions (CO2eq) (AR5)") | 
                                      (Agri_emis_cntry["element"] == "Emissions (N2O)") |
                                        (Agri_emis_cntry["element"] == "Emissions (CO2)") |
                                      (Agri_emis_cntry["element"] == "Emissions (CH4)")|
                                     (Agri_emis_cntry["element"] == "Emissions (F-gases)"), :]
Agri_em_actual.reset_index(drop = True, inplace = True)
Rename_Agri_em_actual = Agri_em_actual.rename(columns = {"area": "Country", "value":"Glb_Agri_Emission_ktonnes", "element":"GH-gases", "item": "Agri_Industry_Sector" })
Org_agri_emi_act = Rename_Agri_em_actual[["year", "Country", "GH-gases", "Agri_Industry_Sector", "Glb_Agri_Emission_ktonnes", "unit"]]
del Org_agri_emi_act['unit']
final_agri_em_act_df = Org_agri_emi_act.loc[(Org_agri_emi_act["Glb_Agri_Emission_ktonnes"]>0), :]
final_agri_em_act_df.reset_index (drop =True, inplace = True)
final_agri_em_act_df= final_agri_em_act_df.sort_values("year")
final_agri_em_act_df
               

,year,Country,GH-gases,Agri_Industry_Sector,Glb_Agri_Emission_ktonnes
91168,1990,United Arab Emirates,Emissions (CO2),Farm-gate emissions,3256.9086
89735,1990,Syrian Arab Republic,Emissions (CO2),Food Household Consumption,504.7280
89761,1990,Viet Nam,Emissions (CO2eq) (AR5),IPPU,1310.2623
13053,1990,Sweden,Emissions (CH4),All sectors with LULUCF,571.0949
43371,1990,Portugal,Emissions (N2O),All sectors with LULUCF,14.4681
...,...,...,...,...,...
1381,2019,Chile,Emissions (CH4),Food Household Consumption,6.0095
84582,2019,Sweden,Emissions (CH4),Energy - food systems,20.6405
112260,2019,South Africa,Emissions (CO2eq) (AR5),LULUCF,6753.3552
96583,2019,Pakistan,Emissions (CH4),Food Packaging,0.2140


In [18]:
final_agri_em_act_df.count()

year                         115763
Country                      115763
GH-gases                     115763
Agri_Industry_Sector         115763
Glb_Agri_Emission_ktonnes    115763
dtype: int64

### Using Postgresql to populate the tables in sSQL with data 

In [21]:
from config import password

In [22]:
protocol = 'postgresql'
username = 'postgres'
host = 'localhost'
port = 5432
database_name = 'Global_Emissions'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
# connection = engine.connect()

In [23]:
engine.table_names()

['Global_oil_data_iea',
 'Global_oil_demand',
 'Global_Greenhouse_Emissions',
 'Global_Agri_Greenhouse_Emissions',
 'Global_Agri_Percent_Emissions']

### Current writing to SQL is set at "replace" - use "append" to add to SLQ database for additional countries

In [24]:
final_agri_em_act_df.to_sql(name='Global_Agri_Greenhouse_Emissions', con=engine, if_exists='replace', index=False)

In [25]:
final_agri_em_per_df.to_sql(name='Global_Agri_Percent_Emissions', con=engine, if_exists='replace', index=False)

In [26]:
pd.read_sql_query('select * from "Global_Agri_Greenhouse_Emissions"', con=engine).head()

,year,Country,GH-gases,Agri_Industry_Sector,Glb_Agri_Emission_ktonnes
0,1990,United Arab Emirates,Emissions (CO2),Farm-gate emissions,3256.9086
1,1990,Syrian Arab Republic,Emissions (CO2),Food Household Consumption,504.7280
2,1990,Viet Nam,Emissions (CO2eq) (AR5),IPPU,1310.2623
3,1990,Sweden,Emissions (CH4),All sectors with LULUCF,571.0949
4,1990,Portugal,Emissions (N2O),All sectors with LULUCF,14.4681


In [27]:
pd.read_sql_query('select * from "Global_Agri_Percent_Emissions"', con=engine).head()

,year,Country,GH-gases,Agri_Industry_Sector,Percent_Glb_Agri_Emission
0,1990,Egypt,Emissions Share (CO2eq) (AR5),Energy,66.7577
1,1990,Japan,Emissions Share (CO2eq) (AR5),All sectors without LULUCF,106.3107
2,1990,Japan,Emissions Share (CH4),All sectors with LULUCF,100.0000
3,1990,Japan,Emissions Share (N2O),Food Packaging,0.2160
4,1990,Peru,Emissions Share (N2O),Food Waste Disposal,1.9951
